In [56]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import time
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from datetime import datetime

In [2]:
DATADIR = ".\\detected_lines"
CATEGORIES = ["w", "a", "d"]
IMG_SIZE = 80

In [3]:
min_images_number = 99999

for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    image_number = len(os.listdir(path))
    if image_number < min_images_number:
        min_images_number = image_number
        
print (min_images_number)

1262


In [4]:
def create_training_data():
    tr_data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        
        class_num = CATEGORIES.index(category)
#         print (class_num)
        
        label = [0, 0, 0]
        label[class_num] = 1
        images_list = os.listdir(path)
        for i in range(min_images_number):
            img = images_list[i]
#             print (img)
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                tr_data.append([new_array, label])
            except Exception as e:
                print (e)
    random.shuffle(tr_data)
    return tr_data

In [5]:
training_data = create_training_data()

X = []
y = []

for img, label in training_data:
    X.append(img)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
print(X[0].shape)
y = np.array(y)

# print(y.shape)

X = X / 255.0

(80, 80, 1)


In [43]:
model = Sequential()
model.add(Conv2D(256, (7, 7), input_shape = X[0].shape, activation = "relu", padding = 'same'  )   )
# model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(256, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.2))

model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.2))

model.add(Dense(32, activation = "relu"))
# model.add(Dropout(0.2))

model.add(Dense(3, activation = "softmax"))

In [44]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 80, 80, 256)       12800     
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 78, 78, 256)       590080    
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 39, 39, 256)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 37, 37, 128)       295040    
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 18, 18, 128)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 16, 16, 64)        73792     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 8, 8, 64)        

In [45]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ['accuracy'])

In [46]:
model.fit(X, y, batch_size = 32, epochs = 10, validation_split = 0.1)

Train on 3407 samples, validate on 379 samples
Epoch 1/10
3407/3407 [==============================] - 29s 9ms/step - loss: 0.4203 - accuracy: 0.8268 - val_loss: 0.3418 - val_accuracy: 0.8997
Epoch 2/10
3407/3407 [==============================] - 29s 8ms/step - loss: 0.2696 - accuracy: 0.8984 - val_loss: 0.1975 - val_accuracy: 0.9367
Epoch 3/10
3407/3407 [==============================] - 29s 8ms/step - loss: 0.2216 - accuracy: 0.9210 - val_loss: 0.1867 - val_accuracy: 0.9314
Epoch 4/10
3407/3407 [==============================] - 29s 8ms/step - loss: 0.1776 - accuracy: 0.9387 - val_loss: 0.1679 - val_accuracy: 0.9340
Epoch 5/10
3407/3407 [==============================] - 29s 8ms/step - loss: 0.1632 - accuracy: 0.9480 - val_loss: 0.1566 - val_accuracy: 0.9446
Epoch 6/10
3407/3407 [==============================] - 29s 8ms/step - loss: 0.1424 - accuracy: 0.9554 - val_loss: 0.1491 - val_accuracy: 0.9499
Epoch 7/10
3407/3407 [==============================] - 29s 8ms/step - loss: 0.1230

In [57]:
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")

In [66]:
NAME = "cnn_model_imgsize" + str(IMG_SIZE) + "-DATE-" + dt_string
model.save(NAME + ".h5")